In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM, Dense, BatchNormalization
from keras.utils import to_categorical
import tensorflow as tf

# --- SETTINGS ---
base_dir = r"C:\Users\zezom\PycharmProjects\HorusEye"
dl_ready_dir = os.path.join(base_dir, "Data", "Processed", "DL_ready")
labels_path = os.path.join(base_dir, "train.csv")

# --- Load and filter labels for MI only ---
labels_df = pd.read_csv(labels_path)
mi_labels = ['Left', 'Right']  # change as needed
mi_labels_df = labels_df[labels_df['label'].isin(mi_labels)]

X = []
y = []
missing_files = []

for i, row in mi_labels_df.iterrows():
    subject = row['subject_id']
    session = row['trial_session']
    trial_num = int(row['trial'])  # 1-based
    label = row['label']

    npy_path = os.path.join(dl_ready_dir, f"{subject}_{session}_EEGdata_preprocessed_DLready.npy")
    if not os.path.exists(npy_path):
        missing_files.append(npy_path)
        continue

    epochs = np.load(npy_path)  # shape: (n_trials, n_channels, n_samples)

    if trial_num - 1 >= epochs.shape[0]:
        print(f"Warning: trial number {trial_num} out of range in file {npy_path}")
        continue

    X.append(epochs[trial_num - 1])  # Select the trial data
    y.append(label)

X = np.stack(X)  # shape: (num_trials, n_channels, n_samples)

# Encode labels
label_encoder = LabelEncoder()
y_enc = label_encoder.fit_transform(y)

# Transpose X to (samples, timesteps, channels) for Conv1D + LSTM input
X = np.transpose(X, (0, 2, 1))

# One-hot encode labels
num_classes = len(np.unique(y_enc))
y_cat = to_categorical(y_enc, num_classes=num_classes)

# Split into train and validation sets with stratification
X_train, X_val, y_train, y_val = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y_enc
)

# Build the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=7, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))

model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    ]
)

if missing_files:
    print(f"Warning: {len(missing_files)} files were missing, e.g. {missing_files[:3]}")

val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_acc:.4f}")

train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"Training Accuracy: {train_acc:.4f}")


C:\Users\zezom\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 2244, 64)       │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2244, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1122, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1122, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1118, 128)      │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1118, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 559, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 559, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 557, 256)       │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 557, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 278, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 278, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 348,354 (1.33 MB)

 Trainable params: 347,458 (1.33 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 52s 592ms/step - accuracy: 0.5111 - loss: 0.7178 - val_accuracy: 0.4619 - val_loss: 0.6993
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 56s 689ms/step - accuracy: 0.4959 - loss: 0.7125 - val_accuracy: 0.4992 - val_loss: 0.6977
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 79s 650ms/step - accuracy: 0.4959 - loss: 0.7044 - val_accuracy: 0.4992 - val_loss: 0.6963
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 53s 655ms/step - accuracy: 0.5002 - loss: 0.6981 - val_accuracy: 0.5039 - val_loss: 0.6948
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 50s 620ms/step - accuracy: 0.5064 - loss: 0.6991 - val_accuracy: 0.4743 - val_loss: 0.6966
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 47s 577ms/step - accuracy: 0.5259 - loss: 0.6946 - val_accuracy: 0.4728 - val_loss: 0.6957
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 49s 609ms/step - accuracy: 0.5084 - loss: 0.6957 - val_accuracy: 0.5023 - val_loss: 0.6962
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 50s 612ms/step - accuracy: 0.4990 - loss: 0.6960 - val_accu

In [4]:
import os
import pickle

# --- Save MI model and label encoder ---
mi_model_path = os.path.join(base_dir, "mi_model.keras")
mi_encoder_path = os.path.join(base_dir, "mi_label_encoder.pkl")

# Save the trained model (recommended Keras format)
model.save(mi_model_path)
print(f"MI model saved to: {mi_model_path}")

# Save the LabelEncoder object (for later decoding)
with open(mi_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)
print(f"MI label encoder saved to: {mi_encoder_path}")


MI model saved to: C:\Users\zezom\PycharmProjects\HorusEye\mi_model.keras
MI label encoder saved to: C:\Users\zezom\PycharmProjects\HorusEye\mi_label_encoder.pkl
